# Starter: Zero shot data extraction using LLMs

In [3]:
import pandas as pd
import json
import ast
from openai import OpenAI
import vllm
import json_metric

/home/bluepurple/Git/ZeroShotDataExtraction/json_metric.py:57: SyntaxWarning: invalid escape sequence '\['
  $$


IndentationError: unexpected indent (json_metric.py, line 145)

In [8]:
llm = vllm.LLM(
    model='meta-llama/Llama-3.2-3B-Instruct',
    tensor_parallel_size=1, 
    gpu_memory_utilization=0.80, 
    max_model_len=8192,
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    #distributed_executor_backend="ray",
)
# tokenizer = llm.get_tokenizer()

INFO 08-17 19:03:02 [__init__.py:235] Automatically detected platform cuda.
WARNING 08-17 19:03:08 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-17 19:03:08 [config.py:1604] Using max model len 8192
INFO 08-17 19:03:10 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 08-17 19:03:10 [core.py:572] Waiting for init message from front-end.
INFO 08-17 19:03:10 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=Deco

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-17 19:03:18 [default_loader.py:262] Loading weights took 2.96 seconds
INFO 08-17 19:03:18 [gpu_model_runner.py:1892] Model loading took 6.0160 GiB and 3.946249 seconds
INFO 08-17 19:03:20 [gpu_worker.py:255] Available KV cache memory: 5.55 GiB
INFO 08-17 19:03:20 [kv_cache_utils.py:833] GPU KV cache size: 51,936 tokens
INFO 08-17 19:03:20 [kv_cache_utils.py:837] Maximum concurrency for 8,192 tokens per request: 6.34x
INFO 08-17 19:03:21 [core.py:193] init engine (profile, create kv cache, warmup model) took 2.63 seconds


## prompts

In [9]:
system_prompt = """As a medical professional, please analyze the following medical notes carefully and extract relevant information in the specified JSON format.
Do not include any values that are not mentioned in the notes."""

In [10]:
# Define the template with a placeholder
prompt_template = """
For each entry:
- Accurately identify and extract patient data, visit motivation, symptoms, and vital signs.
- Only include keys that have corresponding information in the notes, omitting any keys that are not mentioned.
- Follow this JSON structure:

```json
{
  "patient_info": {
    "age": <number>,
    "gender": <string>
  },
  "visit_motivation": "",
  "symptoms": [<string>],
  "vital_signs": {
    "heart_rate": {
      "value": <number>,
      "unit": "bpm"
    },
    "oxygen_saturation": {
      "value": <number>,
      "unit": "%"
    },
    "cholesterol_level": {
      "value": <number>,
      "unit": "mg/dL"
    },
    "glucose_level": {
      "value": <number>,
      "unit": "mg/dL"
    },
    "temperature": {
      "value": <number>,
      "unit": "°C"
    },
    "respiratory_rate": {
      "value": <number>,
      "unit": "breaths/min"
    },
    "blood_pressure": {
      "systolic": {
        "value": <number>,
        "unit": "mmHg"
      },
      "diastolic": {
        "value": <number>,
        "unit": "mmHg"
      }
    }
  }
}

Here are the clinical notes to review:

{medical_notes}

"""

In [11]:
# load test data
test_df = pd.read_csv("medical-note-extraction-h-2-o-gen-ai-world-ny/test.csv")
train_df = pd.read_csv("medical-note-extraction-h-2-o-gen-ai-world-ny/train.csv")

In [12]:
test_df.head()

,ID,Note
0,10823,**Clinical Notes**\n\n**Patient Information:**...
1,10824,**Clinical Notes**\n\n**Patient Information:**...
2,10825,Clinical Note:\n\nPatient: 71-year-old male\nC...
3,10826,**Clinical Notes**\n\n**Patient Information:**...
4,10827,**Clinical Notes**\n\n**Patient Information:**...


In [13]:
train_df.shape

(10822, 3)

In [15]:
# Try a small subset
trial_df = test_df.iloc[:10].copy()

In [20]:
#Build prompts (assemble the system_prompt, prompt template, raw medical notes for every patient)
all_prompts = []
for index,row in train_df.iterrows():
    
    medical_notes = row.Note
    filled_prompt = f"{prompt_template}".replace("{medical_notes}", medical_notes)    
    message=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": filled_prompt}
    ]
    all_prompts.append(message)
    

In [21]:
print(all_prompts[0])

[{'role': 'system', 'content': 'As a medical professional, please analyze the following medical notes carefully and extract relevant information in the specified JSON format.\nDo not include any values that are not mentioned in the notes.'}, {'role': 'user', 'content': '\nFor each entry:\n- Accurately identify and extract patient data, visit motivation, symptoms, and vital signs.\n- Only include keys that have corresponding information in the notes, omitting any keys that are not mentioned.\n- Follow this JSON structure:\n\n```json\n{\n  "patient_info": {\n    "age": <number>,\n    "gender": <string>\n  },\n  "visit_motivation": "",\n  "symptoms": [<string>],\n  "vital_signs": {\n    "heart_rate": {\n      "value": <number>,\n      "unit": "bpm"\n    },\n    "oxygen_saturation": {\n      "value": <number>,\n      "unit": "%"\n    },\n    "cholesterol_level": {\n      "value": <number>,\n      "unit": "mg/dL"\n    },\n    "glucose_level": {\n      "value": <number>,\n      "unit": "mg/d

In [22]:
def print_outputs(outputs):
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Generated text: {generated_text!r}")
        print("-" * 80)

## zero shot prediction

In [23]:
%%time
from vllm import SamplingParams
sampling_params = SamplingParams(n=1,
                                 temperature=0.2, 
                                 top_p=0.9,
                                 seed=777, # Seed for reprodicibility
                                 max_tokens=1024,)

from time import time
start = time()

# responses = llm.generate(
#     all_prompts[0],
#     sampling_params,
#     use_tqdm = True
# )

outputs = llm.chat(messages=all_prompts,
                   sampling_params=sampling_params,
                   use_tqdm=True)

end = time()
elapsed = (end-start)/60. #minutes
print(f"Inference took {elapsed} minutes!")

INFO 08-17 19:26:23 [chat_utils.py:473] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|                     | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

Inference took 0.47876977125803627 minutes!
CPU times: user 522 ms, sys: 160 ms, total: 683 ms
Wall time: 28.7 s


In [24]:
# Batched (many at once)
assert isinstance(all_prompts, list) and isinstance(all_prompts[0], list)
outputs = llm.chat(messages=all_prompts, sampling_params=sampling_params)
assert len(outputs) == len(all_prompts)  # one result per conversation


Adding requests:   0%|          | 0/10 [00:00<?, ?it/s]

Processed prompts:   0%|                     | 0/10 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.…

In [29]:
print_outputs(outputs)

Generated text: '```json\n{\n  "patient_info": {\n    "age": 48,\n    "gender": "Male"\n  },\n  "visit_motivation": "Unspecified",\n  "symptoms": [\n    "Symptoms: Increased Blood Pressure"\n  ],\n  "vital_signs": {\n    "heart_rate": {\n      "value": 0,\n      "unit": "bpm"\n    },\n    "oxygen_saturation": {\n      "value": 0,\n      "unit": "bpm"\n    },\n    "cholesterol_level": {\n      "value": 0,\n      "unit": "mg/dL"\n    },\n    "glucose_level": {\n      "value": 0,\n      "unit": "mg/dL"\n    },\n    "temperature": {\n      "value": 0,\n      "unit": "mg/dL"\n    },\n    "respiratory_rate": {\n      "value": 0,\n      "unit": "breath"\n    },\n    "blood_pressure": {\n      "systolic": {\n        "value": 0,\n        "unit": "mmHg"\n      },\n      "diastolic": {\n        "value": 0,\n        "unit": "mmHg"\n    },\n    "cholesterol_level": {\n      "value": 0,\n      "unit": "mg/dL"\n    },\n    "symptoms": [\n      "Symptoms: Increased Blood Pressure"\n    },\n    "sympto

## simple cleanup the outputs

In [30]:
def clean_preds(preds:str):
    try:
        preds = preds.split('```json\n')[1].split('\n```')[0]
    except Exception as e:
        
        # print(e)
        try: 
            preds = preds.split('\n\n')[1]
        except Exception as e:
            # print(e)
            preds = preds
 
    return preds

In [31]:
json_list = []

for output in outputs:
    res = output.outputs[0].text

    
    try:
        json_list.append(json.loads(clean_preds(res)))
    except:
        print("Error")
        json_list.append(res)
    
trial_df['json'] = json_list
sub_df = trial_df[['ID','json']]
# test_trial_df.to_csv("test_with_json2_final.csv", index=False)

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [34]:
sub_df.shape

(10, 2)

In [36]:
sub_df.head()

,ID,json
0,10823,"```json\n{\n ""patient_info"": {\n ""age"": 48..."
1,10824,"```json\n{\n ""patient_info"": {\n ""age"": 53..."
2,10825,"```json\n{\n ""patient_info"": {\n ""age"": 71..."
3,10826,"```json\n{\n ""patient_info"": {\n ""age"": 73..."
4,10827,"```json\n{\n ""patient_info"": {\n ""age"": 48..."


In [32]:
# take a look at a prediction
sub_df.iloc[2]['json']

'```json\n{\n  "patient_info": {\n    "age": 71,\n    "gender": "Male"\n  },\n  "visit_motivation": "Influenza-like symptoms",\n  "symptoms": [\n    "Influenced" ],\n  "symptoms": [\n    "Influenced" ],\n  "symptoms": [\n    "In the medical field" ],\n  "visit_motivation": "In the medical field" ],\n  "visit_motivation": "In the medical field" ],\n  "visit_motivation": "In the medical field" },\n  "patient_info": {\n    "age": 70,\n    "gender": "Male"\n  },\n  "visit_motivation": "The patient presented with symptoms of" ],\n  "symptoms": [\n    "The patient presented with symptoms of" ],\n  "vital_signs": {\n    "heart_rate": {\n    "value": 70,\n    "unit": "bpm" },\n  "temperature": {\n    "value": 37.8,\n    "unit": "°C" },\n  "cholesterol_level": {\n    "value": 70,\n    "unit": "mg/dL" },\n  "symptoms": [\n    "The patient presented with symptoms of" ],\n  "vital_signs": {\n    "heart_rate": {\n    "value": 70,\n    "unit": "bpm" },\n    "oxygen_saturation": {\n    "value": 70,\n